In [11]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, kendalltau

def load_matrices_from_csv(file_paths):
    matrices = []
    matrix_names = []
    for file_path in file_paths:
        df = pd.read_csv(file_path, header=None)  # 读取CSV文件
        matrices.append(df.values)  # 转换为NumPy数组
        matrix_names.append(file_path.split('/')[-1])  # 提取文件名
    return matrices, matrix_names

def calculate_correlation(matrix_list, method='pearson'):
    num_matrices = len(matrix_list)
    correlations = np.zeros((num_matrices, num_matrices))
    
    for i in range(num_matrices):
        for j in range(i, num_matrices):
            if method == 'pearson':
                corr = np.corrcoef(matrix_list[i].flatten(), matrix_list[j].flatten())[0, 1]
            elif method == 'spearman':
                corr, _ = spearmanr(matrix_list[i].flatten(), matrix_list[j].flatten())
            elif method == 'kendall':
                corr, _ = kendalltau(matrix_list[i].flatten(), matrix_list[j].flatten())
            correlations[i, j] = corr
            correlations[j, i] = corr  # 相关性矩阵是对称的
            
    return correlations

def load_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("CSV files", "*.csv")])
    if file_paths:
        global matrix_list, matrix_names
        matrix_list, matrix_names = load_matrices_from_csv(file_paths)
        matrix_display.config(state=tk.NORMAL)
        matrix_display.delete(1.0, tk.END)  # 清空文本框
        matrix_display.insert(tk.END, "已加载的矩阵:\n")
        for name in matrix_names:
            matrix_display.insert(tk.END, f"- {name}\n")
        matrix_display.config(state=tk.DISABLED)

def calculate_correlation_from_loaded_files(method):
    if matrix_list:
        correlation_matrix = calculate_correlation(matrix_list, method=method)
        matrix_display.config(state=tk.NORMAL)
        matrix_display.delete(1.0, tk.END)  # 清空文本框
        matrix_display.insert(tk.END, f"{method.capitalize()} 相关性矩阵:\n")
        for i, name_i in enumerate(matrix_names):
            for j, name_j in enumerate(matrix_names):
                matrix_display.insert(tk.END, f"({name_i}, {name_j}): {correlation_matrix[i, j]:.4f}\n")
        matrix_display.config(state=tk.DISABLED)
        # 保存按钮启用
        save_button.config(state=tk.NORMAL)
    else:
        matrix_display.config(state=tk.NORMAL)
        matrix_display.delete(1.0, tk.END)
        matrix_display.insert(tk.END, "请先加载矩阵文件")
        matrix_display.config(state=tk.DISABLED)
        # 保存按钮禁用
        save_button.config(state=tk.DISABLED)

def save_correlation_matrix():
    if matrix_list:
        file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")], initialfile="1_矩阵相关性.csv")
        if file_path:
            correlation_matrix = calculate_correlation(matrix_list, method='pearson')
            df = pd.DataFrame(correlation_matrix, index=matrix_names, columns=matrix_names)
            df.to_csv(file_path)
            print(f"相关性矩阵已保存到: {file_path}")
    else:
        print("请先加载矩阵文件")

def close_program():
    root.destroy()

# 创建主窗口
root = tk.Tk()
root.title("矩阵相关性计算")

# 设置窗口大小
root.geometry('800x600')  # 可以根据需要调整窗口大小

# 创建和放置按钮
button_frame = tk.Frame(root)
button_frame.pack(pady=5)

load_button = tk.Button(button_frame, text="加载CSV文件", command=load_files)
load_button.grid(row=0, column=0, padx=5)

pearson_button = tk.Button(button_frame, text="计算皮尔逊相关性", command=lambda: calculate_correlation_from_loaded_files('pearson'))
pearson_button.grid(row=0, column=1, padx=5)

spearman_button = tk.Button(button_frame, text="计算斯皮尔曼相关性", command=lambda: calculate_correlation_from_loaded_files('spearman'))
spearman_button.grid(row=0, column=2, padx=5)

kendall_button = tk.Button(button_frame, text="计算肯德尔相关性", command=lambda: calculate_correlation_from_loaded_files('kendall'))
kendall_button.grid(row=0, column=3, padx=5)

save_button = tk.Button(button_frame, text="保存相关性矩阵", command=save_correlation_matrix, state=tk.DISABLED)
save_button.grid(row=0, column=4, padx=5)

close_button = tk.Button(button_frame, text="关闭", command=close_program)
close_button.grid(row=0, column=5, padx=5)

# 创建文本框用于显示加载的矩阵
matrix_display = scrolledtext.ScrolledText(root, width=100, height=25, wrap=tk.WORD, state=tk.DISABLED)
matrix_display.pack(pady=10)

matrix_list = []
matrix_names = []

# 运行主事件循环
root.mainloop()
